### Using this notebook to edit borehole stratigraphic logs collected by the MGS for the purpose of interpolation into 3D subsurface voxel maps in ArcGIS

In [18]:
#import necessary package dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
# load in the appropriate dataset extracted from ArcGIS
data = pd.read_csv('Bear_Spring_Well_strat.csv')
df = pd.DataFrame(data)
#display(df)
#df.to_csv('copy of_'+ 'Bear_Spring_Well_strat.csv') #saves a file copy of the dataframe
df_copy = df.copy()
display(df_copy)

,OBJECTID *,shape *,c5st_seq_no,relateid,depth_top,depth_bot,drllr_desc,color,hardness,strat,...,first_bdrk,last_strat,ohtopunit,ohtopelev,ohbotunit,ohbotelev,botholelev,aquifer,X,Y
0,10356,Point Z,4143193,626647,0,10,CLAY,YELLOW,SOFT,QCUY,...,OGPC,OPDC,OPDC,867.0,OPDC,825,825,OPDC,557629.9810,4869556.294
1,10359,Point Z,4143194,626647,10,30,LIME,YELLOW,HARD,OGPC,...,OGPC,OPDC,OPDC,867.0,OPDC,825,825,OPDC,557629.9810,4869556.294
2,10355,Point Z,4143195,626647,30,160,SHALE,BLUE,HARD,OGDP,...,OGPC,OPDC,OPDC,867.0,OPDC,825,825,OPDC,557629.9810,4869556.294
3,10358,Point Z,4143196,626647,160,260,SANDROCK,BLUE,MEDIUM,OSTP,...,OGPC,OPDC,OPDC,867.0,OPDC,825,825,OPDC,557629.9810,4869556.294
4,10362,Point Z,4143197,634228,0,10,CLAY,YELLOW,SOFT,QCUY,...,OGPR,OPDC,OPDC,933.0,OPDC,890,890,OPDC,560329.0470,4869566.944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,7801,Point Z,6616273,817783,140,145,LIMESTONE,GREEN,MEDIUM,ODCR,...,OGPR,OSTP,OSTP,1055.0,OSTP,990,990,OSTP,557102.0781,4867266.984
187,7810,Point Z,6616274,817783,145,160,LIMESTONE,GREEN,MEDIUM,OPVL,...,OGPR,OSTP,OSTP,1055.0,OSTP,990,990,OSTP,557102.0781,4867266.984
188,7806,Point Z,6616275,817783,160,164,SANDSTONE,TAN,SOFT,OPVL,...,OGPR,OSTP,OSTP,1055.0,OSTP,990,990,OSTP,557102.0781,4867266.984
189,7808,Point Z,6616276,817783,164,175,SANDSTONE,TAN,SOFT,OGWD,...,OGPR,OSTP,OSTP,1055.0,OSTP,990,990,OSTP,557102.0781,4867266.984


In [20]:
#Find and remove the unecessary columns from the dataframe
col_list = list(df_copy)
#print(col_list)
df_copy.drop(['drllr_desc','color','hardness','lith_prim','lith_sec', 'lith_minor',
             'wellid', 'concat', 'stratcode_gen', 'stratcode_detail','thickness',
             'elevation', 'bdrkelev', 'depth2bdrk','first_bdrk','last_strat','ohtopunit','ohtopelev','ohbotelev',
            'botholelev','depth_top','depth_bot','ohbotunit'],axis=1, inplace=True)
display(df_copy)

,OBJECTID *,shape *,c5st_seq_no,relateid,strat,elev_top,elev_bot,aquifer,X,Y
0,10356,Point Z,4143193,626647,QCUY,1245,1235,OPDC,557629.9810,4869556.294
1,10359,Point Z,4143194,626647,OGPC,1235,1215,OPDC,557629.9810,4869556.294
2,10355,Point Z,4143195,626647,OGDP,1215,1085,OPDC,557629.9810,4869556.294
3,10358,Point Z,4143196,626647,OSTP,1085,985,OPDC,557629.9810,4869556.294
4,10362,Point Z,4143197,634228,QCUY,1290,1280,OPDC,560329.0470,4869566.944
...,...,...,...,...,...,...,...,...,...,...
186,7801,Point Z,6616273,817783,ODCR,1115,1110,OSTP,557102.0781,4867266.984
187,7810,Point Z,6616274,817783,OPVL,1110,1095,OSTP,557102.0781,4867266.984
188,7806,Point Z,6616275,817783,OPVL,1095,1091,OSTP,557102.0781,4867266.984
189,7808,Point Z,6616276,817783,OGWD,1091,1080,OSTP,557102.0781,4867266.984


In [21]:
print(len(df_copy))

191


In [22]:
#basis for working code
#Iterate over the rows and delete the rows that don't have "OGSC", "OGPR","OGCM", or "ODCR" in the "strat" column

display(df_copy)

# using .iterrows()
col = 4
count = 0
df_filtered= pd.DataFrame()
accepted_units = {"OGSC", "OGPR","OGCM", "ODCR"}
for i in range(len(df_copy)):
    if df_copy.iloc[i][col] in accepted_units:
        print("Found in index:" + str(i) +'and in column '+ str(col))
        df_filtered[i] = df_copy.iloc[i]
        count+= 1
print(count)

,OBJECTID *,shape *,c5st_seq_no,relateid,strat,elev_top,elev_bot,aquifer,X,Y
0,10356,Point Z,4143193,626647,QCUY,1245,1235,OPDC,557629.9810,4869556.294
1,10359,Point Z,4143194,626647,OGPC,1235,1215,OPDC,557629.9810,4869556.294
2,10355,Point Z,4143195,626647,OGDP,1215,1085,OPDC,557629.9810,4869556.294
3,10358,Point Z,4143196,626647,OSTP,1085,985,OPDC,557629.9810,4869556.294
4,10362,Point Z,4143197,634228,QCUY,1290,1280,OPDC,560329.0470,4869566.944
...,...,...,...,...,...,...,...,...,...,...
186,7801,Point Z,6616273,817783,ODCR,1115,1110,OSTP,557102.0781,4867266.984
187,7810,Point Z,6616274,817783,OPVL,1110,1095,OSTP,557102.0781,4867266.984
188,7806,Point Z,6616275,817783,OPVL,1095,1091,OSTP,557102.0781,4867266.984
189,7808,Point Z,6616276,817783,OGWD,1091,1080,OSTP,557102.0781,4867266.984


Found in index:5and in column 4
Found in index:13and in column 4
Found in index:19and in column 4
Found in index:20and in column 4
Found in index:25and in column 4
Found in index:30and in column 4
Found in index:36and in column 4
Found in index:38and in column 4
Found in index:42and in column 4
Found in index:43and in column 4
Found in index:48and in column 4
Found in index:49and in column 4
Found in index:56and in column 4
Found in index:62and in column 4
Found in index:63and in column 4
Found in index:64and in column 4
Found in index:69and in column 4
Found in index:70and in column 4
Found in index:77and in column 4
Found in index:84and in column 4
Found in index:90and in column 4
Found in index:92and in column 4
Found in index:98and in column 4
Found in index:100and in column 4
Found in index:105and in column 4
Found in index:106and in column 4
Found in index:112and in column 4
Found in index:118and in column 4
Found in index:119and in column 4
Found in index:120and in column 4
Foun

In [23]:
#transpose and adjust index scaling on df_filtered
df_filtered = df_filtered.transpose()
df_filtered = df_filtered.reset_index()
display(df_filtered)

,index,OBJECTID *,shape *,c5st_seq_no,relateid,strat,elev_top,elev_bot,aquifer,X,Y
0,5,10364,Point Z,4143198,634228,OGPR,1280,1270,OPDC,560329.047,4869566.944
1,13,8444,Point Z,4173355,641651,ODCR,1147,1104,OPSH,556999.9811,4868091.009
2,19,7990,Point Z,4524766,107657,OGSC,1280,1130,OSTP,557816.0,4867594.0
3,20,7993,Point Z,4524767,107657,ODCR,1130,1090,OSTP,557816.0,4867594.0
4,25,11397,Point Z,4526796,148347,ODCR,1169,1124,OSTP,559376.0,4870064.0
5,30,10339,Point Z,4529088,220607,ODCR,1148,1102,OPDC,557797.0,4869547.0
6,36,11375,Point Z,4529099,220609,OGPR,1238,1219,MTPL,557546.0,4870055.0
7,38,11372,Point Z,4529101,220609,ODCR,1153,1114,MTPL,557546.0,4870055.0
8,42,7855,Point Z,4529109,220611,OGPR,1248,1208,OSTP,557369.32,4867293.78
9,43,7856,Point Z,4529110,220611,ODCR,1147,1105,OSTP,557369.32,4867293.78


In [7]:
#add a column with a numerical value assigned to each of the stratigraphic units
#1=Stewartville("OGSC")
#2=Prosser("OGPR")
#3=Cummingsville("OGCM")
#4=Decorach("ODCR")

#create and populate an array with the appropriate values
strat_num =np.zeros(52, dtype='int')
col = 5
for i in range(len(df_filtered)):
    if df_filtered.iloc[i][col] == "OGSC":
        strat_num[i] = 1
    elif df_filtered.iloc[i][col] == "OGPR":
        strat_num[i] = 2
    elif df_filtered.iloc[i][col] == "OGCM":
        strat_num[i] = 3
    elif df_filtered.iloc[i][col] == "ODCR":
        strat_num[i] = 4
display(strat_num)

#add this new array of numbers into the existing pandas dataframe df_filtered
df_filtered['strat_num'] = strat_num.tolist()
display(df_filtered)

array([2, 4, 1, 4, 4, 4, 2, 4, 2, 4, 3, 4, 4, 1, 3, 4, 1, 4, 4, 4, 2, 4,
       2, 4, 1, 4, 4, 2, 3, 4, 4, 4, 1, 4, 3, 2, 2, 2, 3, 4, 4, 2, 2, 2,
       3, 2, 2, 2, 3, 3, 4, 4])

,index,OBJECTID *,shape *,c5st_seq_no,relateid,strat,elev_top,elev_bot,aquifer,X,Y,strat_num
0,5,10364,Point Z,4143198,634228,OGPR,1280,1270,OPDC,560329.047,4869566.944,2
1,13,8444,Point Z,4173355,641651,ODCR,1147,1104,OPSH,556999.9811,4868091.009,4
2,19,7990,Point Z,4524766,107657,OGSC,1280,1130,OSTP,557816.0,4867594.0,1
3,20,7993,Point Z,4524767,107657,ODCR,1130,1090,OSTP,557816.0,4867594.0,4
4,25,11397,Point Z,4526796,148347,ODCR,1169,1124,OSTP,559376.0,4870064.0,4
5,30,10339,Point Z,4529088,220607,ODCR,1148,1102,OPDC,557797.0,4869547.0,4
6,36,11375,Point Z,4529099,220609,OGPR,1238,1219,MTPL,557546.0,4870055.0,2
7,38,11372,Point Z,4529101,220609,ODCR,1153,1114,MTPL,557546.0,4870055.0,4
8,42,7855,Point Z,4529109,220611,OGPR,1248,1208,OSTP,557369.32,4867293.78,2
9,43,7856,Point Z,4529110,220611,ODCR,1147,1105,OSTP,557369.32,4867293.78,4


In [12]:
#convert elevation data from feet to meters
#df_filtered['elev_top'] = df_filtered['elev_top'] / 3.28084
#df_filtered['elev_bot'] = df_filtered['elev_bot'] / 3.28084

df_filtered.loc[:, 'elev_top'] /= 3.28084

df_filtered.iloc[:, 6] = df_filtered.iloc[:, 5]
#df['four'] = df['four']*5

IndexingError: Too many indexers

In [8]:
#save the dataframe as a csv file
df_filtered.to_csv('Filtered_Well_Strat.csv')

In [9]:
#test cells
print(len(df_copy))
df_copy.columns.get_loc("strat")
df_copy.iloc[0][df_copy.columns.get_loc("strat")]
df_copy.iloc[1][1]
df_copy.iloc[0]

191


OBJECTID *           10356
shape *            Point Z
c5st_seq_no        4143193
relateid            626647
strat                 QCUY
elev_top              1245
elev_bot              1235
aquifer               OPDC
X               557629.981
Y              4869556.294
Name: 0, dtype: object